### cv2.findContours()  :
findContours(InputOutputArray image,  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;OutputArrayOfArrays contours,  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;OutputArray hierarchy,  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;int mode,  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;int method,  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Point offset = Point())  

image:输入图像，图像必须为8-bit单通道图像（一般为灰度图或者是二值图）  
contours:检测到的轮廓，每个轮廓都是以点向量的形式进行存储,即使用point类型的vector表示  
hierarchy:可选的变量，包含了图像的拓扑信息  
mode:轮廓检索模式  

RETR_EXTERNAL:表示只检测最外层轮廓  
RETR_LIST:提取所有轮廓，并放置在list中，检测的轮廓不建立等级关系   
RETR_CCOMP:提取所有轮廓，并将轮廓组织成双层结构(two-level hierarchy),顶层为连通域的外围边界，次层位内层边界   
RETR_TREE:提取所有轮廓并重新建立网状轮廓结构  
RETR_FLOODFILL:洪水填充法  

method:轮廓近似方法  
CHAIN_APPROX_NONE：获取每个轮廓的每个像素，相邻的两个点的像素位置差不超过1  
CHAIN_APPROX_SIMPLE：压缩水平方向，垂直方向，对角线方向的元素，只保留该方向的重点坐标，如果一个矩形轮廓只需4个点来保存轮廓信息  
CHAIN_APPROX_TC89_L1和CHAIN_APPROX_TC89_KCOS使用Teh-Chinl链逼近算法中的一种  

offset:轮廓点可选偏移量，有默认值Point()

### cv2.Canny():  
edge = cv2.Canny(image, threshold1, threshold2)  
image:需要处理的原图像，该图像必须为单通道的灰度图；  
threshold1:是阈值1；  
threshold2:是阈值2。  
强度梯度大于maxVal的任何边缘肯定是边缘，低于minVal的边缘肯定是非边缘，因此被丢弃。

In [6]:
import cv2
import numpy as np

KNOWN_DISTANCE = 60  #已知距离（标定图），单位：英寸
KNOWN_WIDTH = 11.69  #已知A4纸的宽度，单位：英寸
KNOWN_HEIGHT = 8.26  #已知A4纸的高度，单位：英寸

# 计算A4纸到摄像头的距离
def distance_to_camera(knownWidth, focalLength, pixelWidth):
    return (knownWidth*focalLength) / pixelWidth;

# 检测A4纸
def A4_Detect(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image_gray = cv2.GaussianBlur(image_gray, (5,5), 0)
    edges = cv2.Canny(image_gray, 0, 50)
    
    (cnts, _) = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    c = max(cnts, key = cv2.contourArea)
    
    return cv2.minAreaRect(c)

In [7]:
# 存储图像路径
IMAGE_PATHS1 = ["0-20-0.jpg", "0-30-0.jpg", "0-50-0.jpg", "0-60-0.jpg"]
IMAGE_PATHS2 = ["1-20-1.jpg", "1-30-1.jpg", "1-50-1.jpg", "1-60-1.jpg"]

In [8]:
# 利用已知的A4纸与已知的距离标定焦距
standard_image = cv2.imread(IMAGE_PATHS1[3])
known_A4 = A4_Detect(standard_image)

focalLength = (known_A4[1][0]*KNOWN_DISTANCE) / KNOWN_WIDTH

In [9]:
print("焦距:",focalLength,"inchs")

焦距: 3009.7179037572178 inchs


In [10]:
# 读取未知的图像，并估量A4纸到摄像头的距离
test_image = cv2.imread(IMAGE_PATHS1[2])
unknown_A4 = A4_Detect(test_image)

distance = distance_to_camera(KNOWN_WIDTH, focalLength, unknown_A4[1][0])

In [11]:
print("估量距离为：",distance,"inchs")

估量距离为： 50.45164101176325 inchs


==================================================这是一条华丽丽的分割线=====================================================

In [8]:
# standard_image = cv2.imread(IMAGE_PATHS[0])
r = 700.0 / standard_image.shape[1]
dim = (700, int(standard_image.shape[0] * r))

for images in IMAGE_PATHS2:
    image = cv2.imread(images)
    
    # 1.转为灰度图，以便后续调用cv2.findContours()
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    image_gray_resized = cv2.resize(image_gray,dim,interpolation = cv2.INTER_AREA)
    cv2.imshow("image_gray",image_gray_resized)
        
    # 2.使用高斯模糊去噪点
    image_blur = cv2.GaussianBlur(image_gray, (47,47), 0)
    image_gray_resized = cv2.resize(image_blur,dim,interpolation = cv2.INTER_AREA)
    cv2.imshow("image_blur",image_gray_resized)
    
    # 3.使用Canny算子检测轮廓
    edges = cv2.Canny(image_blur, 100, 200)
    edges_resized = cv2.resize(edges,dim,interpolation = cv2.INTER_AREA)
    cv2.imshow("image_Canny",edges_resized)
    
    # 4.在原图上画出能框住A4纸的最小矩形
    cnt = A4_Detect(image)
    box = cv2.boxPoints(cnt)
    box = np.int0(box)
    cv2.drawContours(image,[box],0,(0,0,255),10)
    original_resized = cv2.resize(image,dim,interpolation = cv2.INTER_AREA) 
    cv2.imshow("image_original",original_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()